# Bell State Fidelity Measurement

## Method

We measure fidelity with the ideal Bell state |Φ⁺⟩_L = (|0⟩_L|0⟩_L + |1⟩_L|1⟩_L)/√2 using:

**F = (1 + ⟨X̄X̄⟩ - ⟨Ȳ Ȳ⟩ + ⟨Z̄Z̄⟩) / 4**

Where ⟨P̄₁P̄₂⟩ is the expectation value of logical Pauli operators on Alice and Bob's qubits.

## Why This Works

Perfect Bell state has:
- ⟨X̄X̄⟩ = +1 (X-basis correlation)
- ⟨Ȳ Ȳ⟩ = -1 (Y-basis anti-correlation)
- ⟨Z̄Z̄⟩ = +1 (Z-basis correlation)

This gives F = (1 + 1 - (-1) + 1) / 4 = 1.0

## Measurement Procedure

For each correlation ⟨P̄₁P̄₂⟩:

1. **Basis rotation**: Apply gates to measure in correct basis
   - X: Apply H
   - Y: Apply S†, H
   - Z: No rotation

2. **Measure**: All 14 physical qubits in computational basis

3. **Compute logical outcome**: For [[7,1,3]] Steane code, logical operators are transversal
   ```
   Logical outcome = (sum of 7 measurements) mod 2
   Even parity → |0⟩ → +1
   Odd parity → |1⟩ → -1
   ```

4. **Calculate expectation**: Average over many shots
   ```
   ⟨P̄₁P̄₂⟩ = mean(alice_eigenvalue × bob_eigenvalue)
   ```

## Algorithm

```
1. Measure ⟨X̄X̄⟩: H on all 14 qubits → measure → compute parity
2. Measure ⟨Ȳ Ȳ⟩: S†, H on all 14 qubits → measure → compute parity  
3. Measure ⟨Z̄Z̄⟩: Measure directly → compute parity
4. Calculate: F = (1 + ⟨XX⟩ - ⟨YY⟩ + ⟨ZZ⟩) / 4
```

## Key Properties

- **Efficient**: 3 measurements vs 16 for full tomography
- **Transversal gates**: Logical Paulis = physical Paulis on all 7 qubits
- **Exact**: Formula gives true fidelity for Bell states
- **Standard**: Widely used in quantum communication

In [1]:
import stim
import numpy as np


def encode_steane_713(circuit, qubits):
    q = qubits
    circuit.append("H", [q[4], q[5], q[6]])
    circuit.append("CX", [q[0], q[1]])
    circuit.append("CX", [q[0], q[2]])
    circuit.append("CX", [q[6], q[3]])
    circuit.append("CX", [q[6], q[1]])
    circuit.append("CX", [q[6], q[0]])
    circuit.append("CX", [q[5], q[3]])
    circuit.append("CX", [q[5], q[2]])
    circuit.append("CX", [q[5], q[0]])
    circuit.append("CX", [q[4], q[3]])
    circuit.append("CX", [q[4], q[2]])
    circuit.append("CX", [q[4], q[1]])


def logical_bell_pair():
    c = stim.Circuit()
    alice, bob = list(range(7)), list(range(7, 14))
    c.append("H", [0])

    encode_steane_713(c, alice)
    for q in alice:
        c.append("DEPOLARIZE1", [q], 0.00)
    encode_steane_713(c, bob)
    for i in range(7):
        c.append("CX", [alice[i], bob[i]])
    return c


def measure_correlation(circuit, alice_op, bob_op, shots=10000):
    """Measure ⟨alice_op ⊗ bob_op⟩"""
    c = circuit.copy()
    
    # Basis rotations
    for qubits, op in [(range(7), alice_op), (range(7, 14), bob_op)]:
        if op == 'X':
            for q in qubits:
                c.append("H", [q])
        elif op == 'Y':
            for q in qubits:
                c.append("S_DAG", [q])
                c.append("H", [q])
    
    # Measure
    for i in range(14):
        c.append("M", [i])
    
    measurements = c.compile_sampler().sample(shots)
    alice_parity = np.sum(measurements[:, :7], axis=1) % 2
    bob_parity = np.sum(measurements[:, 7:14], axis=1) % 2
    
    return np.mean((1 - 2 * alice_parity) * (1 - 2 * bob_parity))


def bell_fidelity(circuit, shots=10000):
    """Calculate fidelity with Bell state |Φ⁺⟩ = (|00⟩ + |11⟩)/√2"""
    xx = measure_correlation(circuit, 'X', 'X', shots)
    yy = measure_correlation(circuit, 'Y', 'Y', shots)
    zz = measure_correlation(circuit, 'Z', 'Z', shots)
    return (1 + xx - yy + zz) / 4


# Usage
circuit = logical_bell_pair()
fidelity = bell_fidelity(circuit, shots=50000)
print(f"Fidelity: {fidelity:.4f}")

Fidelity: 1.0000


In [8]:
import stim
import numpy as np


def encode_steane_713(circuit, qubits):
    q = qubits
    circuit.append("H", [q[4], q[5], q[6]])
    circuit.append("CX", [q[0], q[1]])
    circuit.append("CX", [q[0], q[2]])
    circuit.append("CX", [q[6], q[3]])
    circuit.append("CX", [q[6], q[1]])
    circuit.append("CX", [q[6], q[0]])
    circuit.append("CX", [q[5], q[3]])
    circuit.append("CX", [q[5], q[2]])
    circuit.append("CX", [q[5], q[0]])
    circuit.append("CX", [q[4], q[3]])
    circuit.append("CX", [q[4], q[2]])
    circuit.append("CX", [q[4], q[1]])


def logical_bell_pair():
    c = stim.Circuit()
    alice, bob = list(range(7)), list(range(7, 14))
    c.append("H", [0])

    encode_steane_713(c, alice)
    for q in alice:
        c.append("DEPOLARIZE1", [q], 0.00)
    encode_steane_713(c, bob)
    # for i in range(7):
    #     c.append("CX", [alice[i], bob[i]])
    return c


def measure_correlation(circuit, alice_op, bob_op, shots=10000):
    """Measure correlation between alice_op and bob_op"""
    c = circuit.copy()
    
    # Basis rotations
    for qubits, op in [(range(7), alice_op), (range(7, 14), bob_op)]:
        if op == 'X':
            for q in qubits:
                c.append("H", [q])
        elif op == 'Y':
            for q in qubits:
                c.append("S_DAG", [q])
                c.append("H", [q])
    
    # Measure
    for i in range(14):
        c.append("M", [i])
    
    measurements = c.compile_sampler().sample(shots)
    alice_parity = np.sum(measurements[:, :7], axis=1) % 2
    bob_parity = np.sum(measurements[:, 7:14], axis=1) % 2
    
    return np.mean((1 - 2 * alice_parity) * (1 - 2 * bob_parity))


def measure_single_qubit_expectation(circuit, qubit_index, pauli_op, shots=10000):
    """Measure expectation value of a Pauli operator on a single qubit"""
    c = circuit.copy()
    
    # Apply basis rotation
    if pauli_op == 'X':
        c.append("H", [qubit_index])
    elif pauli_op == 'Y':
        c.append("S_DAG", [qubit_index])
        c.append("H", [qubit_index])
    # Z basis needs no rotation
    
    # Measure all qubits
    for i in range(c.num_qubits):
        c.append("M", [i])
    
    measurements = c.compile_sampler().sample(shots)
    qubit_measurements = measurements[:, qubit_index]
    
    # Convert 0/1 to +1/-1 eigenvalues
    eigenvalues = 1 - 2 * qubit_measurements
    return np.mean(eigenvalues)


def compute_single_qubit_density_matrix(circuit, qubit_index, shots=10000):
    """
    Compute the 2x2 density matrix for a single qubit.
    
    Args:
        circuit: stim.Circuit to sample from
        qubit_index: Index of the qubit to compute density matrix for
        shots: Number of measurement samples per basis
    
    Returns:
        2x2 numpy array representing the density matrix
    """
    # Measure Pauli expectations
    exp_x = measure_single_qubit_expectation(circuit, qubit_index, 'X', shots)
    exp_y = measure_single_qubit_expectation(circuit, qubit_index, 'Y', shots)
    exp_z = measure_single_qubit_expectation(circuit, qubit_index, 'Z', shots)
    
    # Pauli matrices
    sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
    sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
    sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)
    identity = np.eye(2, dtype=complex)
    
    # Reconstruct density matrix: rho = (I + <X>*sigma_x + <Y>*sigma_y + <Z>*sigma_z) / 2
    rho = (identity + exp_x * sigma_x + exp_y * sigma_y + exp_z * sigma_z) / 2
    
    return rho


def measure_two_qubit_expectation(circuit, qubit1, qubit2, pauli_op1, pauli_op2, shots=10000):
    """Measure expectation value of pauli_op1 on qubit1 and pauli_op2 on qubit2"""
    c = circuit.copy()
    
    # Apply basis rotations
    for qubit, op in [(qubit1, pauli_op1), (qubit2, pauli_op2)]:
        if op == 'X':
            c.append("H", [qubit])
        elif op == 'Y':
            c.append("S_DAG", [qubit])
            c.append("H", [qubit])
    
    # Measure all qubits
    for i in range(c.num_qubits):
        c.append("M", [i])
    
    measurements = c.compile_sampler().sample(shots)
    meas1 = measurements[:, qubit1]
    meas2 = measurements[:, qubit2]
    
    # Convert to eigenvalues
    eigenvalues = (1 - 2 * meas1) * (1 - 2 * meas2)
    return np.mean(eigenvalues)


def compute_two_qubit_density_matrix(circuit, qubit1, qubit2, shots=10000):
    """
    Compute the 4x4 density matrix for two qubits.
    
    Args:
        circuit: stim.Circuit to sample from
        qubit1: Index of first qubit
        qubit2: Index of second qubit
        shots: Number of measurement samples per basis
    
    Returns:
        4x4 numpy array representing the density matrix
    """
    # Pauli matrices
    sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
    sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
    sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)
    sigma_i = np.eye(2, dtype=complex)
    
    paulis = {'I': sigma_i, 'X': sigma_x, 'Y': sigma_y, 'Z': sigma_z}
    
    # Measure all 16 correlations
    rho = np.zeros((4, 4), dtype=complex)
    
    for op1_name, op1_matrix in paulis.items():
        for op2_name, op2_matrix in paulis.items():
            if op1_name == 'I' and op2_name == 'I':
                expectation = 1.0  # Identity always has expectation 1
            elif op1_name == 'I':
                expectation = measure_single_qubit_expectation(circuit, qubit2, op2_name, shots)
            elif op2_name == 'I':
                expectation = measure_single_qubit_expectation(circuit, qubit1, op1_name, shots)
            else:
                expectation = measure_two_qubit_expectation(circuit, qubit1, qubit2, op1_name, op2_name, shots)
            
            # Add contribution to density matrix
            rho += expectation * np.kron(op1_matrix, op2_matrix)
    
    # Normalize
    rho = rho / 4
    
    return rho


def bell_fidelity(circuit, shots=10000):
    """Calculate fidelity with Bell state |Phi+> = (|00> + |11>)/sqrt(2)"""
    xx = measure_correlation(circuit, 'X', 'X', shots)
    yy = measure_correlation(circuit, 'Y', 'Y', shots)
    zz = measure_correlation(circuit, 'Z', 'Z', shots)
    return (1 + xx - yy + zz) / 4


# Usage examples
circuit = logical_bell_pair()

# Original fidelity calculation
fidelity = bell_fidelity(circuit, shots=50000)
print(f"Logical Bell Pair Fidelity: {fidelity:.4f}")

# Example 1: Get density matrix for Alice's first qubit
print("\n" + "="*60)
print("Alice's Qubit 0 Density Matrix (2x2):")
print("="*60)
rho_alice_0 = compute_single_qubit_density_matrix(circuit, qubit_index=0, shots=10000)
print(rho_alice_0)
print(f"Trace: {np.trace(rho_alice_0):.6f}")
print(f"Purity: {np.trace(rho_alice_0 @ rho_alice_0):.6f}")

# Example 2: Get density matrix for Bob's first qubit
print("\n" + "="*60)
print("Bob's Qubit 7 Density Matrix (2x2):")
print("="*60)
rho_bob_7 = compute_single_qubit_density_matrix(circuit, qubit_index=7, shots=10000)
print(rho_bob_7)
print(f"Trace: {np.trace(rho_bob_7):.6f}")
print(f"Purity: {np.trace(rho_bob_7 @ rho_bob_7):.6f}")

# Example 3: Get density matrix for two qubits (Alice's qubit 0 and Bob's qubit 7)
print("\n" + "="*60)
print("Two-Qubit Density Matrix (4x4) for qubits 0 and 7:")
print("="*60)
rho_two_qubit = compute_two_qubit_density_matrix(circuit, qubit1=0, qubit2=7, shots=10000)
print(rho_two_qubit)
print(f"Trace: {np.trace(rho_two_qubit):.6f}")
print(f"Purity: {np.trace(rho_two_qubit @ rho_two_qubit):.6f}")

Logical Bell Pair Fidelity: 0.2503

Alice's Qubit 0 Density Matrix (2x2):
[[0.5006+0.j     0.0037-0.0006j]
 [0.0037+0.0006j 0.4994+0.j    ]]
Trace: 1.000000+0.000000j
Purity: 0.500029+0.000000j

Bob's Qubit 7 Density Matrix (2x2):
[[0.5017+0.j    0.0055-0.004j]
 [0.0055+0.004j 0.4983+0.j   ]]
Trace: 1.000000+0.000000j
Purity: 0.500098+0.000000j

Two-Qubit Density Matrix (4x4) for qubits 0 and 7:
[[ 2.492e-01+0.j      -1.500e-04+0.0006j   5.200e-03-0.00185j
  -2.150e-03-0.00155j]
 [-1.500e-04-0.0006j   2.496e-01+0.j      -4.250e-03-0.00285j
  -3.600e-03-0.00325j]
 [ 5.200e-03+0.00185j -4.250e-03+0.00285j  2.532e-01+0.j
   6.500e-04-0.0023j ]
 [-2.150e-03+0.00155j -3.600e-03+0.00325j  6.500e-04+0.0023j
   2.480e-01+0.j     ]]
Trace: 1.000000+0.000000j
Purity: 0.250202+0.000000j


In [11]:
for i in range(0,7):
    rho_alice_0 = compute_single_qubit_density_matrix(circuit, qubit_index=i, shots=10000)
    print(rho_alice_0)
    
print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
for i in range(7,14):
    rho_alice_0 = compute_single_qubit_density_matrix(circuit, qubit_index=i, shots=10000)
    print(rho_alice_0)

[[0.5065+0.j     0.0018-0.0078j]
 [0.0018+0.0078j 0.4935+0.j    ]]
[[ 0.4948+0.j     -0.002 -0.0059j]
 [-0.002 +0.0059j  0.5052+0.j    ]]
[[0.5084+0.j 0.0035+0.j]
 [0.0035+0.j 0.4916+0.j]]
[[0.5017+0.j     0.0055-0.0004j]
 [0.0055+0.0004j 0.4983+0.j    ]]
[[ 0.4891+0.j     -0.002 -0.0108j]
 [-0.002 +0.0108j  0.5109+0.j    ]]
[[ 0.5034+0.j    -0.0025-0.006j]
 [-0.0025+0.006j  0.4966+0.j   ]]
[[ 0.4934+0.j     -0.0016-0.0057j]
 [-0.0016+0.0057j  0.5066+0.j    ]]
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
[[0.5084+0.j     0.0008+0.0085j]
 [0.0008-0.0085j 0.4916+0.j    ]]
[[0.4882+0.j    0.0086+0.008j]
 [0.0086-0.008j 0.5118+0.j   ]]
[[ 0.4991+0.j     -0.0052-0.0011j]
 [-0.0052+0.0011j  0.5009+0.j    ]]
[[ 0.4945+0.j     -0.0057-0.0002j]
 [-0.0057+0.0002j  0.5055+0.j    ]]
[[0.5034+0.j     0.0014-0.0011j]
 [0.0014+0.0011j 0.4966+0.j    ]]
[[0.5004+0.j     0.0072-0.0029j]
 [0.0072+0.0029j 0.4996+0.j    ]]
[[0.5013+0.j     0.0026-0.0027j]
 [0.0026+0.0027j 0.4987+0.j    ]]
